Решение соревнования https://cleancodecup.ru/problem/promopurchase

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_log_error, accuracy_score
import optuna
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Binarizer
from imblearn.under_sampling import CondensedNearestNeighbour
from tqdm import tqdm

c:\Users\alexa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cls_oct = pd.read_csv('clients_promo_october.csv')
mobile_events = pd.read_csv('mobile_events.csv')
orders = pd.read_csv('orders.csv')
train = pd.read_csv('train_target.csv')
test = pd.read_csv('test.csv')

Как выглядят датасеты:

In [4]:
cls_oct.head()

,ClientUUId,Id,LocalBeginDate,LocalEndDate,OrderType,OrderPrice,Discount
0,000D3A20F23EA95811E7B7104AC23CD3,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,25
1,000D3A20F23EA95811E7B7D79FAABAA1,5,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,300
2,000D3A20F23EA95811E7B8BD71C73877,7,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,200
3,000D3A20F23EA95811E7B99F3ED09FC8,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,30
4,000D3A20F23EA95811E7BD373E79565E,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,20


In [5]:
mobile_events.head()

,ClientUUId,VisitToken,EventName,Platform,Timestamp
0,000D3A22FA54A81611EB315CF5443815,976627AD-76DC-41D9-981E-F2A6CA14B3A9,screen_menu,ios,2023-10-29T16:14:21.343Z
1,2A4A64AF719E994511EE3F2D8B31E188,5b22e610-c71e-4b6a-a842-71e8575ece87,screen_menu,android,2023-10-29T11:04:36.4840000Z
2,000D3AAC977BBB2F11ECDD081F5481AE,BDCEB72F-2B6A-4565-801A-6CB729279C6F,open_app,ios,2023-10-29T11:14:34.723Z
3,000D3A21DA51A81411EAE468856F96F1,A66E2DA6-4E2E-4C37-9FBF-627732623212,add_to_cart,ios,2023-10-29T14:03:08.363Z
4,000D3A23B0DC80D811E67F4D0A9CA345,1BE8139F-A3D3-439F-A51B-5CBD4D3FFF1B,screen_menu,ios,2023-10-29T08:35:40.127Z


In [6]:
orders.head()

,OrderUUId,addressId,deliverySectorId,ClientUUId,Date,SaleDate,UnitUUId,NewClient,ClientOrderNumber,ProductUUId,CategoryId,ProductTotalPrice,MenuPrice,OrderState,OrderPaymentType,OrderTotalPrice,OrderType,apply_promo
0,020DD76CCD949AE011EDE031ED5D76FD,NaN,NaN,000D3AAC9DCABB2E11EBE0614C89D044,2023-04-21,2023-04-21T20:51:26.000Z,000D3A2480C380DA11E692CFB9F92484,0,18,000D3A240C71BE9A11E719BE2AB264A6,4,1.0,45.0,4,2,1.0,3,0
1,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EC9AABAD354BE051CCF1EE1F83A8D0,1,485.0,485.0,4,2,1164.0,1,0
2,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EB846583CB3BDEA80D1A7F2E184D40,1,589.0,589.0,4,2,1164.0,1,0
3,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB267FD,4,45.0,45.0,4,2,1164.0,1,0
4,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB264A6,4,45.0,45.0,4,2,1164.0,1,0


In [7]:
train.head()

,ClientUUId,Id,OrderType,LocalBeginDate,LocalEndDate,OrderPrice,Discount,apply_promo
0,000D3A20F23EA95811E7B99F3ED09FC8,7,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,200,0
1,000D3A20F23EA95811E7B99F3ED09FC8,6,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,25,0
2,000D3A20F23EA95811E7BD373E79565E,5,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,799,200,0
3,000D3A20F23EA95811E7BD373E79565E,5,"1,2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,1249,250,0
4,000D3A20F23EA95811E7BDAE71CD3F33,6,"2,3",2023-11-02T00:00:00.000Z,2023-11-05T23:59:00.000Z,699,30,0


In [9]:
test.head()

,ClientUUId,Id,OrderType,LocalBeginDate,LocalEndDate,OrderPrice,Discount
0,000D3A20F23EA95811E7C0A95563344E,7,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200
1,000D3A20F23EA95811E7C7892A0CE261,5,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,699,200
2,000D3A20F23EA95811E7CD686C396528,6,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,20
3,000D3A20F23EA95911E7CEA8C574EDAE,5,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200
4,000D3A20F23EA95911E7D4F05C59C978,7,"2,3",2023-11-02T00:00:00Z,2023-11-05T23:59:00Z,799,200


In [274]:
def get_df(train, orders, mobile_events, cls_oct, test=False):
    
    ord = orders[['ClientUUId', 'OrderUUId', 'OrderTotalPrice', 'apply_promo']].drop_duplicates()
    nn = pd.read_csv('new_col.csv') #Датасет, агреггированный с clients_promo_october.csv
    
    sas = pd.merge(ord.groupby('ClientUUId')[['OrderTotalPrice', 'apply_promo']].sum().reset_index(), nn, on='ClientUUId')
    sas = pd.merge(ord.ClientUUId.value_counts(), sas, on='ClientUUId')

    oks = pd.get_dummies(cls_oct.OrderType + ',' + cls_oct.Discount.apply(lambda x: str(x)) + ',' + cls_oct.OrderPrice.apply(lambda x: str(x)), dtype=int)
    oks = pd.concat((cls_oct.ClientUUId, oks), axis=1)
    oks = oks.groupby('ClientUUId').sum().reset_index()

    new_train = train.drop(['LocalBeginDate', 'LocalEndDate'], axis=1)
    new_train = pd.get_dummies(new_train, columns=['Id', 'OrderType'], dtype=int)
    mobile_events_ = pd.concat((mobile_events, pd.get_dummies(mobile_events.EventName, dtype=int)), axis=1).drop(['EventName'], axis=1)
    ev1 = mobile_events_.groupby(['ClientUUId'])[['apply_personal_offer', 'create_order', 'open_bonusaction']].sum().reset_index()
    ev2 = mobile_events_.groupby(['ClientUUId'])[['apply_personal_offer', 'create_order', 'open_bonusaction']].mean().reset_index()
    ev3 = mobile_events_.groupby(['ClientUUId'])[['apply_personal_offer', 'create_order', 'open_bonusaction']].count().reset_index()
    ev = pd.merge(pd.merge(ev1, ev2, on='ClientUUId'), ev3, on='ClientUUId')
    X = pd.merge(new_train, sas, on='ClientUUId', how='left')
    X = pd.merge(X, oks, on='ClientUUId', how='left')
    X = pd.merge(X, ev, on='ClientUUId', how='left').drop(['ClientUUId'], axis=1)
    if not test:
        y = X.apply_promo_x
        X = X.drop(['apply_promo_x'], axis=1).fillna(0)
        return X.to_numpy(), y.to_numpy()
    else:
        return X.fillna(0).to_numpy()

Извлекаем признаки из данных. Бинаризуем размеры скидок и порог срабатывания промокода, помимо всего прочего извлекаем из истории предыдущих покупок суммы покупок, основанные на них признаки, а также из истории использования приложением его интерес к промокодам и стремление заказать.

In [246]:
X, y = get_df(train, orders, mobile_events, cls_oct, test=False)
X_test = get_df(test, orders, mobile_events, cls_oct, test=True)

In [276]:
params = {'learning_rate': 0.02585593198584217, 'max_depth': 10, 'subsample': 0.5102954089862447, 'colsample_bytree': 0.5066879444200845, 'gamma': 0.008018887885891779, 
          'reg_alpha': 0.13779374848933656, 'reg_lambda': 2.5602362510574066, 'n_estimators': 987}

model = XGBClassifier(**params)

X1, X2, y1, y2 = train_test_split(X, y, test_size=0.2, random_state=14)
X1_, y1_ = RandomUnderSampler(sampling_strategy='majority', random_state=42).fit_resample(X1, y1)

model.fit(X1_, y1_)
y1_pred = model.predict(X1)
y2_pred = model.predict(X2)
print(roc_auc_score(y1, y1_pred))
print(roc_auc_score(y2, y2_pred))
print(confusion_matrix(y1, y1_pred))
print(confusion_matrix(y2, y2_pred))

0.8589600742804085
0.7661534371843266
[[15464  6076]
 [    0   652]]
[[3874 1500]
 [  33  142]]


In [242]:
def objective(trial):

    params = {
    'objective': 'reg:squarederror',
    'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
    'max_depth': trial.suggest_int('max_depth', 3, 10),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'gamma': trial.suggest_float('gamma', 0, 5),
    'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
    'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
    }

    model = xgb.train(params, X_train, num_boost_round=trial.suggest_int('n_estimators', 100, 1000))

    y_pred = model.predict(X_test)

    return roc_auc_score(y2, y_pred)

In [ ]:
scal = StandardScaler()
X1 = scal.fit_transform(X1)
X_train = xgb.DMatrix(X1, label=y1)
X_test = xgb.DMatrix(X2, label=y2)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best Parameters:', study.best_params)
print('Best ROC_AUC:', study.best_value)

Trial 21 finished with value: 0.7630240842150035 and parameters: {'learning_rate': 0.02585593198584217, 'max_depth': 10, 'subsample': 0.5102954089862447, 'colsample_bytree': 0.5066879444200845, 'gamma': 0.008018887885891779, 
          'reg_alpha': 0.13779374848933656, 'reg_lambda': 2.5602362510574066, 'n_estimators': 987}

In [278]:
X, y = get_df(train, orders, mobile_events, cls_oct, test=False)
X_test = get_df(test, orders, mobile_events, cls_oct, test=True)

In [279]:
params = {'learning_rate': 0.02585593198584217, 'max_depth': 10, 'subsample': 0.5102954089862447, 'colsample_bytree': 0.5066879444200845, 'gamma': 0.008018887885891779, 
          'reg_alpha': 0.13779374848933656, 'reg_lambda': 2.5602362510574066, 'n_estimators': 987}

model = XGBClassifier(**params)
X1, y1 = RandomUnderSampler(sampling_strategy='majority', random_state=42).fit_resample(X, y)

In [ ]:
model.fit(X1, y1)

In [281]:
pred_test = model.predict_proba(X_test)[:, 1]

In [282]:
result = pd.concat((pd.read_csv('test.csv'), pd.DataFrame(pred_test, columns=['apply_promo'])), axis=1)

In [284]:
result.to_csv('submission.csv', index=False)

Результирующая метрика = 72342558,80

11/37 место